In [1]:
import pandas as pd
import numpy as np
import os

In [4]:
def calculate_power(window):
    power = {}
    for axis in ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']:
        power[axis] = (window[axis]**2).mean()
    return power

def calculate_energy(window):
    energy = {}
    for axis in ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']:
        energy[axis] = (window[axis]**2).sum()
    return energy

In [5]:
from scipy.stats import entropy, skew, kurtosis

def calculate_features(window):
    features = {}
    for axis, axis_name in zip(['x', 'y', 'z'], ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']):
        # Статистичні ознаки
        features[f'mean_{axis}'] = window[axis_name].mean()
        features[f'std_{axis}'] = window[axis_name].std()
        features[f'var_{axis}'] = window[axis_name].var()
        features[f'max_{axis}'] = window[axis_name].max()
        features[f'min_{axis}'] = window[axis_name].min()

        # Envelope Metrics
        features[f'rms_{axis}'] = np.sqrt(np.mean(window[axis_name]**2))

        # Коефіцієнт асиметрії (Skewness)
        features[f'skew_{axis}'] = skew(window[axis_name])

        # Ексцес (Kurtosis)
        features[f'kurt_{axis}'] = kurtosis(window[axis_name])

        # Крос-кореляція між різними осями
        features['corr_xy'] = window['accelerometer_X'].corr(window['accelerometer_Y'])
        features['corr_xz'] = window['accelerometer_X'].corr(window['accelerometer_Z'])
        features['corr_yz'] = window['accelerometer_Y'].corr(window['accelerometer_Z'])

        # Розрахунок потужності та енергії
        power = calculate_power(window)
        energy = calculate_energy(window)

        for axis in ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']:
            features[f'power_{axis}'] = power[axis]
            features[f'energy_{axis}'] = energy[axis]
    
            # Ентропія сигналу
            # Ентропія вимагає нормалізованих імовірностей, тут проста апроксимацію !
            hist_counts, _ = np.histogram(window[axis_name], bins=10, density=True)
            features[f'entropy_{axis}'] = entropy(hist_counts)

    # Сигнальна магнітуда
    sma = window[['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']].abs().sum().sum()
    features['sma'] = sma

    return features

In [2]:
def read_files_from_folder(folder_path):
    all_files = os.listdir(folder_path)
    data_frames = []

    for file in all_files:
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            data_frames.append(df)

    return pd.concat(data_frames, ignore_index=True)

In [3]:
def get_folder_names(parent_directory):
    items = os.listdir(parent_directory)
    folders = [item for item in items if os.path.isdir(os.path.join(parent_directory, item))]
    return folders

parent_directory = 'D:\Projects\Data\homework\data'
folder_names = get_folder_names(parent_directory)
print(folder_names)

activity_dataframes = {}

for folder_name in folder_names:
    folder_path = os.path.join(parent_directory, folder_name)
    activity_dataframes[folder_name] = read_files_from_folder(folder_path)

#print(activity_dataframes)
print(activity_dataframes.keys())
print(type(activity_dataframes))


['idle', 'running', 'stairs', 'walking']
dict_keys(['idle', 'running', 'stairs', 'walking'])
<class 'dict'>


In [6]:
for activity, df in activity_dataframes.items():
    print(f"{activity}: {df.shape}")

idle: (31170, 3)
running: (102240, 3)
stairs: (4950, 3)
walking: (55500, 3)


In [11]:
from sklearn.preprocessing import StandardScaler

window_size = 208  # Розмір вікна у вимірюваннях
overlap = 104      # 50% перекриття

# Створення словника для зберігання вікон для кожної активності
activity_windows_dict = {}

# Цикл по всіх DataFrame у словнику
for activity, df in activity_dataframes.items():
    # Відокремлення числових колонок для нормалізації
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df_numeric = df[numeric_columns]

    # Нормалізація числових даних
    scaler = StandardScaler()
    df_normalized = pd.DataFrame(scaler.fit_transform(df_numeric), columns=df_numeric.columns)

    # Створення вікон
    windows = [df_normalized[j:j + window_size] for j in range(0, len(df_normalized) - window_size + 1, window_size - overlap)]

    # Додавання списку вікон до словника
    activity_windows_dict[activity] = windows

In [12]:
combined_df_list = []

for activity, df in activity_dataframes.items():
    # Поділ даних на вікна та застосування calculate_features
    for i in range(0, len(df) - window_size + 1, window_size - overlap):
        window = df[i:i + window_size]
        features = calculate_features(window)
        features['activity'] = activity  # Додавання мітки активності
        combined_df_list.append(pd.DataFrame([features]))

combined_df = pd.concat(combined_df_list, ignore_index=True)


In [15]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(activity_windows_dict)

{   'idle': [        accelerometer_X  accelerometer_Y  accelerometer_Z
0           1.306595         0.847178        -0.327996
1           0.898266         0.702908        -0.434753
2          -1.454822        -0.889159         0.189973
3           7.230840        -0.736402        -0.458476
4           2.407009        -0.846727         0.229513
..               ...              ...              ...
203         0.247707        -0.758467         0.620955
204         0.393044        -0.790715         0.664449
205         0.455332        -0.809386         0.664449
206         0.538382        -0.816175         0.688173
207         0.434568        -0.799202         0.617002

[208 rows x 3 columns],
                     accelerometer_X  accelerometer_Y  accelerometer_Z
104         0.489936        -0.800899         0.640725
105         0.455332        -0.778835         0.632818
106         0.469174        -0.821267         0.624910
107         0.427648        -0.795808         0.644679
108     

In [16]:
for activity, df in activity_dataframes.items():
    print(f"Активність: {activity}")
    print(df.isna().sum())
    print("\n") 

Активність: idle
accelerometer_X    0
accelerometer_Y    0
accelerometer_Z    0
dtype: int64


Активність: running
accelerometer_X    0
accelerometer_Y    0
accelerometer_Z    0
dtype: int64


Активність: stairs
accelerometer_X    0
accelerometer_Y    0
accelerometer_Z    0
dtype: int64


Активність: walking
accelerometer_X    0
accelerometer_Y    0
accelerometer_Z    0
dtype: int64




In [17]:
print(combined_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858 entries, 0 to 1857
Data columns (total 38 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean_x                   1858 non-null   float64
 1   std_x                    1858 non-null   float64
 2   var_x                    1858 non-null   float64
 3   max_x                    1858 non-null   float64
 4   min_x                    1858 non-null   float64
 5   rms_x                    1858 non-null   float64
 6   skew_x                   1858 non-null   float64
 7   kurt_x                   1858 non-null   float64
 8   corr_xy                  1858 non-null   float64
 9   corr_xz                  1858 non-null   float64
 10  corr_yz                  1858 non-null   float64
 11  power_accelerometer_X    1858 non-null   float64
 12  energy_accelerometer_X   1858 non-null   float64
 13  entropy_accelerometer_X  1858 non-null   float64
 14  power_accelerometer_Y   

In [20]:
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

# Якщо 'y' є категорійним, перетворюємо його в числові мітки
label_encoder = LabelEncoder()
y_numeric = label_encoder.fit_transform(y)

# Приклад візуалізації за допомогою PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_numeric)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Dataset')
plt.show()

ModuleNotFoundError: No module named 'matplotlib.artist'

In [21]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

# Перетворення категорійних міток у числові
label_encoder = LabelEncoder()
y_numeric = label_encoder.fit_transform(y)

# Відображення числових міток на оригінальні категорії
label_mapping = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))

# Візуалізація за допомогою PCA
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# Створення 3D-графіка
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Розсіяна діаграма
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=y_numeric)

# Додавання легенди
legend_labels = [label_mapping[i] for i in range(len(label_mapping))]
ax.legend(handles=scatter.legend_elements()[0], labels=legend_labels, title="Categories")

ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
ax.set_title('3D PCA of Dataset')
plt.show()

ModuleNotFoundError: No module named 'matplotlib.artist'

In [22]:
from sklearn.model_selection import train_test_split

X = combined_df.drop('activity', axis=1)  # Все, крім стовпця активності
y = combined_df['activity']  # Тільки стовпець активності

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(X_train, y_train)

# Оцінка моделі на тестовій вибірці
accuracy = model.score(X_test, y_test)
print(f"Точність моделі: {accuracy}")

Точність моделі: 0.9946236559139785


In [24]:
from sklearn.metrics import precision_score, recall_score

y_pred = model.predict(X_test)

precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')  

print(f"Точність: {precision}")
print(f"Повнота: {recall}")

Точність: 0.9946236559139785
Повнота: 0.9946236559139785


In [25]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_model = SVC(C=5.0, kernel='linear', gamma='scale')

# Тренування моделі на навчальних даних
svm_model.fit(X_train, y_train)

SVC(C=5.0, kernel='linear')

1. C - це параметр регуляризації, який контролює компроміс між створенням гладкої межі рішення і класифікацією всіх тренувальних точок правильно. Нижче значення C спричиняє більш гладку межу рішення, в той час як вище значення дозволяє моделі максимізувати класифікацію тренувальних точок навіть за рахунок можливого перенавчання.  
2. kernel визначає тип ядра, яке використовується SVM. Доступні опції включають 'linear', 'poly' (поліноміальне), 'rbf' (радіально-базисна функція) і 'sigmoid'. Вибір ядра залежить від того, чи є ваші дані лінійно роздільними.  
3. gamma - це параметр для ядер 'rbf', 'poly' і 'sigmoid'. Він визначає, наскільки далеко вплив одного тренувального прикладу сягає. Нижчі значення gamma означають 'ширший' радіус впливу тренувальних точок.  

In [26]:
# Передбачення міток на тестових даних
y_pred = svm_model.predict(X_test)

# Розрахунок метрик
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro')

print(f"Точність моделі: {accuracy}")
print(f"Точність (Precision): {precision}")
print(f"Повнота (Recall): {recall}")

Точність моделі: 0.9865591397849462
Точність (Precision): 0.9458116635462174
Повнота (Recall): 0.9488818389848138
